## Qiskit 101

**Keun Jun Park**  
Graduate, Department of Industrial Engineering  

---

<a href="https://colab.research.google.com/github/AzAINN/Qiskit-Fall-Fest-Purdue-University/blob/main/Qiskit_Basic.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
# Setup
!pip install qiskit
!pip install qiskit-aer
!pip install pylatexenc
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

use_aer_simulator = False

try:
    from qiskit_aer import AerSimulator
    from qiskit import transpile

    backend = AerSimulator()
    use_aer_simulator = True
    print("Using AerSimulator.")
except ImportError:
    from qiskit import Aer, execute

    backend = Aer.get_backend("qasm_simulator")
    print("Using qasm_simulator from Qiskit Aer.")

def run_and_get_counts(circuit, shots=1024):
    if use_aer_simulator:
        tqc = transpile(circuit, backend)
        result = backend.run(tqc, shots=shots).result()
        return result.get_counts()
    else:
        result = execute(circuit, backend=backend, shots=shots).result()
        return result.get_counts()




# The Communication Challenge
<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1gopyEtcl3aekQ2Fqijd9ABpQM6y4JzmK" width="200">
</p>


Alice works at Mission Control on **Earth**.  
Bob is an astronaut on **Mars**.

Sending data between Earth and Mars is:
- expensive
- limited in bandwidth

NASA tells them:

> "You must find a way to communicate using **as few signals as possible**."

Classically, to send 2 bits (00, 01, 10, 11), Alice must send **2 classical bits**.


Alice wonders if **quantum bits — qubits** — can help.



# Qubit

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=19K9ZVWgCZUmtW-jqJ_cOilF3MJmADOTk" width="400">
</p>


A **classical bit** is either 0 or 1.  
A **qubit** can be prepared in state |0⟩ or |1⟩, but later we’ll see it can be much richer than that.

First, let’s just see how a single qubit behaves when we measure it.

In [ ]:
# Qubit demo: measuring |0> and |1>

# Measure |0>
qc_zero = QuantumCircuit(1, 1)
qc_zero.measure(0, 0)

print("Circuit for |0> measurement:")
display(qc_zero.draw("mpl"))
counts_zero = run_and_get_counts(qc_zero, shots=512)
print("Results for |0>:", counts_zero)

# Measure |1>
qc_one = QuantumCircuit(1, 1)
qc_one.x(0)          # flip |0> to |1>
qc_one.measure(0, 0)

print("\nCircuit for |1> measurement:")
display(qc_one.draw("mpl"))
counts_one = run_and_get_counts(qc_one, shots=512)
print("Results for |1>:", counts_one)

# Plot one histogram as example
plot_histogram(counts_one)
plt.show()


# Superposition

Alice now experiments with **superposition** on Earth.

She prepares a qubit in |0⟩ and applies a **Hadamard gate** H, creating:

$
|+\rangle = \frac{|0\rangle + |1\rangle}{\sqrt{2}}
$

This means the qubit is in a *blend* of 0 and 1 at the same time.

She sends this qubit to Bob on Mars.  
Bob measures it…

- Sometimes he gets 0  
- Sometimes he gets 1  
- Over many shots, it’s about 50/50

Superposition is our first hint that qubits are richer than classical bits, even though measurement still gives a single 0 or 1 each run.

**Resource**

IBM Learning Platform

https://quantum.cloud.ibm.com/learning/en/modules/quantum-mechanics/superposition-with-qiskit


In [ ]:
# Superposition demo: |+> = H|0>

qc_plus = QuantumCircuit(1, 1)
qc_plus.h(0)               # create |+>
qc_plus.measure(0, 0)

print("Circuit for |+> superposition:")
display(qc_plus.draw("mpl"))

counts_plus = run_and_get_counts(qc_plus, shots=1024)
print("Measurement results for |+>:", counts_plus)

plot_histogram(counts_plus)
plt.show()


# Entanglement

Before Bob leaves Earth for Mars, Alice and Bob meet in the quantum lab.

They create many **entangled pairs** in the Bell state:

$
|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}
$

For each pair:

- Alice keeps one qubit on **Earth**
- Bob takes the other qubit to **Mars**

These two qubits do not have separate states; they form one shared, entangled state.

If both measure:
- They never see 01 or 10
- They always see 00 or 11

The results are perfectly correlated, even when separated by millions of kilometers.

Let’s build and measure this entangled state.

**Resource**

EPR State/Bell State

https://pennylane.ai/qml/glossary/what-are-bell-states

Spooky Science of Quantum Entanglement

https://science.nasa.gov/what-is-the-spooky-science-of-quantum-entanglement/


In [ ]:
# Entanglement demo: Bell state |Φ+> = (|00> + |11>)/sqrt(2)

qc_bell = QuantumCircuit(2, 2)
qc_bell.h(0)              # superposition on qubit 0
qc_bell.cx(0, 1)          # entangle qubit 1 with qubit 0
qc_bell.measure([0, 1], [0, 1])

print("Circuit for Bell state |Φ+>:")
display(qc_bell.draw("mpl"))

counts_bell = run_and_get_counts(qc_bell, shots=1024)
print("Measurement results for Bell state:", counts_bell)

plot_histogram(counts_bell)
plt.show()


# Superdense Coding — Exercise

Alice and Bob use their shared entanglement to reduce the communication cost between Earth and Mars.

Goal:
- Alice wants to send one of four messages: 00, 01, 10, or 11
- Sending classical bits across space is expensive
- Classically, she must send 2 bits
- With superdense coding, she sends **only 1 qubit**, and Bob still recovers **2 bits**

## Protocol Summary

1. Before the mission:
   Alice and Bob share an entangled pair in the state |Φ⁺⟩.

2. Encoding on Earth (Alice):
   Based on her 2-bit message, Alice applies a gate to her qubit:

   | Message | Gate on Alice's qubit |
   |---------|------------------------|
   | 00      | I (do nothing)         |
   | 01      | X                      |
   | 10      | Z                      |
   | 11      | Z then X               |

3. Transmission:
   Alice sends only **one qubit** to Bob.

4. Decoding on Mars (Bob):
   Bob performs a Bell measurement:
   - CX
   - H
   - measure

Your task:
**Implement the full superdense coding circuit in a function.**

After you finish, run the provided tests to check that your solution correctly encodes and decodes all four possible messages.


In [ ]:
from qiskit import QuantumCircuit

def build_superdense_circuit(message: str) -> QuantumCircuit:
    """
    STUDENT TODO:
    Implement the superdense coding protocol.

    Requirements:
    - Use 2 qubits, 2 classical bits
    - Qubit 1 = Alice's qubit (Earth)
    - Qubit 0 = Bob's qubit (Mars)
    - Step 1: Create Bell pair |Φ+> = (|00> + |11>)/√2
    - Step 2: Alice encodes based on the 2-bit message
    - Step 3: Alice sends qubit 1 to Bob
    - Step 4: Bob decodes using CX + H + measure
    """
    assert message in {"00", "01", "10", "11"}, "Message must be one of: 00, 01, 10, 11"

    qc = QuantumCircuit(2, 2)

    # TODO: Step 1 – Create entangled pair |Φ+⟩

    # TODO: Step 2 – Alice encodes the message using operations on qubit 1

    # TODO: Step 3 – Bob decodes using a Bell measurement

    qc.measure([1, 0], [1, 0])
    return qc


In [ ]:
def test_superdense_coding(build_fn, shots=256):
    messages = ["00", "01", "10", "11"]
    all_passed = True

    print("=== Superdense Coding Test Results ===\n")

    for msg in messages:
        qc = build_fn(msg)
        counts = run_and_get_counts(qc, shots=shots)

        # Take the most likely output
        decoded = max(counts, key=counts.get)

        status = "PASS" if decoded == msg else "FAIL"
        print(f"{status}: sent={msg}, decoded={decoded}, counts={counts}")

        if decoded != msg:
            all_passed = False

    if all_passed:
        print("\n🎉 All tests passed! Your superdense coding implementation is correct.")
    else:
        print("\n⚠️ Some tests failed. Check your encoding or decoding steps.")


In [ ]:
test_superdense_coding(build_superdense_circuit)


# Conclusion — What We Learned
<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1gKBS827tN6plTjMdwfCHs-QwStlj4RgV" width="200">
</p>
In this workshop, we learned the core ideas that make quantum communication powerful:

- A **qubit** can hold richer information than a classical bit.  
- **Superposition** lets a qubit exist in multiple states at once.  
- **Entanglement** creates a shared connection between distant qubits, even across planets.  
- With **superdense coding**, Alice can send **two classical bits** using **only one qubit**.

By combining these ideas, we built a communication protocol that is more efficient than anything classical.

Alice and Bob are happy because they now have a cost-saving, bandwidth-efficient, quantum-powered communication link between Earth and Mars — one that uses fewer transmissions while delivering the same information.

Quantum mechanics helped them solve a real engineering problem, and you just implemented the full protocol yourself!

**Good Resource**

https://quantum.cloud.ibm.com/learning/en
